In [2]:
#set enviroment

import os, sys
os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow.keras.backend as K

import importlib; importlib.reload(K)

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(K)

set_keras_backend("tensorflow")

In [4]:
#dependencies for plotting 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

font = {'size'   : 16}

matplotlib.rc('font', **font)

In [5]:
# seed the pseudorandom number generator
from numpy.random import seed
from numpy.random import rand
# seed random number generator
seed(1)

In [194]:
#Import Tensorflow 

import tensorflow.keras.backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input,  LeakyReLU, Activation
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow.keras.losses
from sklearn.model_selection import train_test_split
from tensorflow.keras import activations
from tensorflow.keras import saving
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization, LayerNormalization
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import GlorotUniform, LecunNormal
from tensorflow.math import square, sqrt
from tensorflow.math import reduce_mean as mean
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [7]:
#read data
nu, theta_e, theta_b, jI, aI, jI_fit, aI_fit, jQ, aQ, rQ, jQ_fit, aQ_fit, rQ_fit, jV,aV, rV, jV_fit, aV_fit, rV_fit= np.loadtxt("data_pol_v2.txt", unpack=True)

In [ ]:
#Data massaging
jQ=-jQ
aQ=-aQ

jQ_fit=-jQ_fit
aQ_fit=-aQ_fit

jV = jV /(theta_b)
aV = aV /(theta_b)
jV_fit = jV/theta_b
aV_fit=aV_fit/theta_b


In [9]:
#Function to check some trivial bounds, we dont want out of bound numbers or zero numbers, this means the coefficients are very small anyway
def check_data(use,absol=False):
    global nu, theta_e, theta_b, jI, aI, jI_fit, aI_fit, jQ, aQ, rQ, jQ_fit, aQ_fit, rQ_fit, jV,aV, rV, jV_fit, aV_fit, rV_fit
    if(absol):
        use=np.abs(use)
    nu=nu[use>1e-299]
    theta_e=theta_e[use>1e-299]
    theta_b=theta_b[use>1e-299]
    
    jI=jI[use>1e-299]
    jI_fit=jI_fit[use>1e-299]

    aI=aI[use>1e-299]
    aI_fit=aI_fit[use>1e-299]

    jQ=jQ[use>1e-299]
    jQ_fit=jQ_fit[use>1e-299]

    aQ=aQ[use>1e-299]
    aQ_fit=aQ_fit[use>1e-299]

    rQ=rQ[use>1e-299]
    rQ_fit=rQ_fit[use>1e-299]

    jV=jV[use>1e-299]
    jV_fit=jV_fit[use>1e-299]

    aV=aV[use>1e-299]
    aV_fit=aV_fit[use>1e-299]

    rV=rV[use>1e-299]
    rV_fit=rV_fit[use>1e-299]


In [ ]:
#Run function on all coefficients and check the shape to see how much changed
print(jI.shape)

check_data(jI)
check_data(aI)

check_data(jQ)
check_data(aQ)

check_data(jV)
check_data(aV)

print(jI.shape)

In [ ]:
#Same for rhoQ and rhoV, but keep an eye on that these are not positive number, so need to take the absolute value
check_data(rQ,absol=True)

check_data(rV,absol=True)
print(jI.shape)

In [ ]:
#Construct an input vector X and output vector y
X = np.zeros(shape=(3,len(nu)))
y = np.zeros(shape=(8,len(jI)))

xmin = np.zeros(shape=(3))
xmax = np.zeros(shape=(3))
ymin = np.zeros(shape=(8))
ymax = np.zeros(shape=(8)) 

#in general we recenter and renormalize all variables so that they have a mean of zero and a range of +/- unity

#first input data
X[0] = np.log10(nu)- 0.5*(np.max(np.log10(nu)) + np.min(np.log10(nu)))
xmin[0]=0.5*(np.max(np.log10(nu)) + np.min(np.log10(nu)))
xmax[0]=np.max(X[0])
X[0]/=np.max(X[0])

print("nu min: ", xmin[0], "max: ", xmax[0])
print("X0 min: ", np.min(X[0]), "max: ", np.max(X[0]))

X[1] = np.log10(theta_e)-0.5*(np.max(np.log10(theta_e)) + np.min(np.log10(theta_e)))
xmin[1]=0.5*(np.max(np.log10(theta_e)) + np.min(np.log10(theta_e)))
xmax[1]=np.max(X[1])
X[1]/=np.max(X[1])

print("theta min: ", xmin[1], "max: ", xmax[1])
print("X1 min: ", np.min(X[1]), "max: ", np.max(X[1]))

xmin[2]=0.5*(np.min(theta_b)+np.max(theta_b))

X[2]=theta_b-xmin[2]
xmax[2]=np.max(X[2])
X[2]/=np.max(X[2])

print("theta_b min: ", xmin[2], "max: ", xmax[2])
print("X2 min: ", np.min(X[2]), "max: ", np.max(X[2]))

#Output data
ymin[0]=0.5*(np.max(np.log10(jI)) + np.min(np.log10(jI)))
y[0] = np.log10(jI) - ymin[0]
ymax[0]=np.max(y[0])
y[0] /=np.max(y[0])

ymin[1]=0.5*(np.max(np.log10(aI)) + np.min(np.log10(aI)))
y[1] = np.log10(aI) - ymin[1]
ymax[1]=np.max(y[1])
y[1] /=np.max(y[1])

ymin[2]=0.5*(np.max(np.log10(jQ)) + np.min(np.log10(jQ)))
y[2] = np.log10(jQ) - ymin[2]
ymax[2]=np.max(y[2])
y[2] /=np.max(y[2])

ymin[3]=0.5*(np.max(np.log10(aQ)) + np.min(np.log10(aQ)))
y[3] = np.log10(aQ) - ymin[3]
ymax[3]=np.max(y[3])
y[3] /=np.max(y[3])

y[4] =  np.sign(rQ)*np.fabs(rQ)**0.25
ymin[4]  = 0.5*(np.max(y[4])+np.min(y[4]))
y[4] = y[4] - ymin[4]
ymax[4]=np.max(y[4])
y[4] /=np.max(y[4])

ymin[5]=0.5*(np.max(np.log10(jV)) + np.min(np.log10(jV)))
y[5] = np.log10(jV) - ymin[5]
ymax[5]=np.max(y[5])
y[5] /=np.max(y[5])

ymin[6]=0.5*(np.max(np.log10(aV)) + np.min(np.log10(aV)))
y[6] = np.log10(aV)- ymin[6]
ymax[6]=np.max(y[6])
y[6] /=np.max(y[6])

y[7] = np.sign(rV)*np.fabs(rV)**0.25
ymin[7]  = 0.5*(np.max(y[7])+np.min(y[7]))
y[7] = y[7] - ymin[7]
ymax[7]=np.max(y[7])
y[7] /=np.max(y[7])

print("Stokes I")
print("min:", ymin[0],"max", ymax[0])
print("min:", ymin[1],"max", ymax[1])

print("Stokes Q")
print("min:", ymin[2],"max", ymax[2])
print("min:", ymin[3],"max", ymax[3])
print("min:", ymin[4], "max", ymax[4])

print("\nStokes V")
print("min:", ymin[5], "max", ymax[5])
print("min:", ymin[6], "max", ymax[6])
print("min:", ymin[7], "max", ymax[7])

print("Stokes I")
print("min:",np.min(y[0]),"max:",np.max(y[0]))
print("min:",np.min(y[1]),"max:",np.max(y[1]))

print("Stokes Q")
print("min:",np.min(y[2]),"max:",np.max(y[2]))
print("min:",np.min(y[3]),"max:",np.max(y[3]))
print("min:",np.min(y[4]),"max:",np.max(y[4]))

print("\nStokes V")
print("min:",np.min(y[5]),"max:",np.max(y[5]))
print("min:",np.min(y[6]),"max:",np.max(y[6]))
print("min:",np.min(y[7]),"max:",np.max(y[7]))

X=X.transpose()
y=y.transpose()
print(X.shape,y.shape)



In [ ]:
#save the recentering and normalization coefficients, we will need them if want to transfer them back.
xrange = [xmin,xmax]
yrange = [ymin,ymax]

out = np.concatenate((xrange, yrange),axis=1)
print(np.shape(out))
out = np.transpose(out)
np.savetxt("minmax.txt",out)

In [ ]:
#time to split training data!  we use 80/20 split between training and validation and keep 20,000 
X_train, X_val, y_train, y_val = train_test_split(X, 
                                                   y,
                                                   test_size=0.2,
                                                   random_state=12412412)

 X_val, X_test, y_val, y_test = train_test_split(X_val, 
                                                     y_val, 
                                                     test_size=0.057414, 
                                                     random_state=124)


print(X.shape,X_train.shape,X_val.shape,X_test.shape)

In [51]:
#set up some intermediate saving callback
checkpoint_filepath = 'best_comb.keras'

model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='mean_absolute_percentage_error',
    save_freq='epoch',
    save_best_only=True)

In [201]:
#Will use exponential learning rate decay
initial_learning_rate = 1e-4
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.95,
    staircase=False)

callback = [model_checkpoint_callback]

#Adam optimizer
opt = Adam(learning_rate=lr_schedule)

#Network consists out of 9 layers, split in sets of 3, containing 100, 200, 300 neurons respectively.
N0 = 8
N1 = 100
N2 = 200
N3 = 300


model = Sequential()

#input vector
model.add(Input(shape=[3]))

#Set 1
model.add(Dense(N1, kernel_initializer=LecunNormal(seed=142),activation='relu'))
model.add(Dense(N1, kernel_initializer=LecunNormal(seed=242),activation='relu'))
model.add(Dense(N1, kernel_initializer=LecunNormal(seed=342),activation='relu'))

#Set 2
model.add(Dense(N2, kernel_initializer=LecunNormal(seed=442),activation='relu'))
model.add(Dense(N2, kernel_initializer=LecunNormal(seed=542),activation='relu'))
model.add(Dense(N2, kernel_initializer=LecunNormal(seed=642),activation='relu'))

#Set 3
model.add(Dense(N3, kernel_initializer=LecunNormal(seed=742),activation='relu'))
model.add(Dense(N3, kernel_initializer=LecunNormal(seed=842),activation='relu'))
model.add(Dense(N3, kernel_initializer=LecunNormal(seed=942),activation='relu'))

#Output Layer
model.add(Dense(N0,activation='linear'))

#Compile the model
model.compile(loss="mean_absolute_error", optimizer=opt,metrics=['mean_absolute_error','mean_absolute_percentage_error'])

In [ ]:
#Time to learn, 1000 epochs, 2000 batchsize
history = model.fit(X_train, y_train, 
                    epochs=1000,
                    batch_size=2000, 
                    validation_data=(X_val, y_val),
                    verbose=2,
                    callbacks=[callback]
                   )

In [61]:
#now we want to save the netork by using keras2cpp. https://github.com/gosha20777/keras2cpp
from keras2cpp import export_model

export_model(model, 'comb_final_compact_v2.model')

In [189]:
#next do a test on the test set we kept aside
predictions = model.predict(X_test,batch_size=1000)

20/20 [==============================] - 0s 1ms/step


In [ ]:
#get stats on training and validations
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']

In [ ]:
# Plot the training and validation loss
plt.semilogy(loss, label='Training loss')
plt.semilogy(val_loss, label='Validation loss')
plt.xlabel("Epoch",fontsize=20)
plt.ylabel("Loss",fontsize=20)
plt.legend()

plt.tight_layout()
plt.savefig("loss.png")

In [ ]:
#Process the input and output data of the test set so we can compare things to the fit functions
X1_test_unscaled = X_test[:,0]
X2_test_unscaled = X_test[:,1] 

nu_test = np.power(10,X1_test_unscaled*xmax[0] + xmin[0])
theta_e_test = np.power(10,X2_test_unscaled*xmax[1] + xmin[1])

jI_predictions_unscaled = predictions[:,0] 
aI_predictions_unscaled=  predictions[:,1] 

jQ_predictions_unscaled = predictions[:,2]
aQ_predictions_unscaled=  predictions[:,3]
rQ_predictions_unscaled = predictions[:,4]

jV_predictions_unscaled = predictions[:,5]
aV_predictions_unscaled=  predictions[:,6]
rV_predictions_unscaled = predictions[:,7]

jI_truth_unscaled = y_test[:,0] 
aI_truth_unscaled = y_test[:,1] 

jQ_truth_unscaled = y_test[:,2] 
aQ_truth_unscaled = y_test[:,3] 
rQ_truth_unscaled = y_test[:,4] 

jV_truth_unscaled = y_test[:,5] 
aV_truth_unscaled = y_test[:,6] 
rV_truth_unscaled = y_test[:,7] 

jI_test = jI_predictions_unscaled
aI_test = aI_predictions_unscaled

jQ_test = jQ_predictions_unscaled
aQ_test = aQ_predictions_unscaled
rQ_test = rQ_predictions_unscaled

jV_test = jV_predictions_unscaled
aV_test = aV_predictions_unscaled
rV_test = rV_predictions_unscaled

jI_truth = jI_truth_unscaled
aI_truth = aI_truth_unscaled

jQ_truth = jQ_truth_unscaled
aQ_truth = aQ_truth_unscaled
rQ_truth = rQ_truth_unscaled

jV_truth = jV_truth_unscaled
aV_truth = aV_truth_unscaled
rV_truth = rV_truth_unscaled

In [191]:
#plotting function
def plot_scatter(ax,data,truth,xlabel,ylabel,textlabel=0,mini=0,maxi=0,xticks=True,yticks=True):
    ax.scatter(data,truth, s=1,c='gray')
    if(mini!=maxi):
        ax.set_xlim(mini,maxi)
        ax.set_ylim(mini,maxi)
    if(textlabel!=0):
        plt.text(.01, .99, textlabel, ha='left', va='top', transform=ax.transAxes,fontsize=30)    
    if(xticks):
        ax.set_xlabel(xlabel)

    if(yticks):
        ax.set_ylabel(ylabel)


In [ ]:
#plot results Network vs ground truth

N=8
font = {'size'   : 25}
print(jI_test.shape)
matplotlib.rc('font', **font)
plt.figure(figsize=(14,14),dpi=100)

ax=plt.subplot(421)
plot_scatter(ax,jI_test,jI_truth,"Network","Truth",r"$j_{\rm I}$",-1,1,xticks=False)
ax=plt.subplot(422)
plot_scatter(ax,aI_test,aI_truth,"Network","Truth",r"$\alpha_{\rm I}$",-1,1,xticks=False,yticks=False)
ax=plt.subplot(423)

plot_scatter(ax,jQ_test,jQ_truth,"Network","Truth",r"$j_{\rm Q}$",-1,1,xticks=False)
ax=plt.subplot(424)

plot_scatter(ax,aQ_test,aQ_truth,"Network","Truth",r"$\alpha_{\rm Q}$",-1,1,xticks=False,yticks=False)
ax=plt.subplot(425)

plot_scatter(ax,rQ_test,rQ_truth,"Network","Truth",r"$\rho_{\rm Q}$",np.min(rQ_test),np.max(rQ_test),xticks=False)
ax=plt.subplot(426)

plot_scatter(ax,jV_test,jV_truth,"Network","Truth",r"$j_{\rm V}$",-1,1,xticks=False,yticks=False)
ax=plt.subplot(427)

plot_scatter(ax,aV_test,aV_truth,"Network","Truth",r"$\alpha_{\rm V}$",-1,1,xticks=True)

ax=plt.subplot(428)

plot_scatter(ax,rV_test,rV_truth,"Network","Truth",r"$\rho_{\rm V}$",np.min(rV_test),np.max(rV_test),xticks=True,yticks=False)

plt.tight_layout()

plt.savefig("trained.png",dpi=100)

In [ ]:
#plot results Fit function vs ground truth

N=8
font = {'size'   : 25}

matplotlib.rc('font', **font)
plt.figure(figsize=(14,14),dpi=100)

ax=plt.subplot(421)
y_fit = (np.log10(jI_fit) - ymin[0])/ymax[0]
y_truth = (np.log10(jI)- ymin[0])/ymax[0]
plot_scatter(ax,y_fit,y_truth,"Fit", "Truth",r"$j_{\rm I}$",-1,1,xticks=False)

ax=plt.subplot(422)
y_fit = (np.log10(aI_fit)- ymin[1])/ymax[1]
y_truth = (np.log10(aI)- ymin[1])/ymax[1]
plot_scatter(ax,y_fit,y_truth,"Fit", "Truth",r"$\alpha_{\rm I}$",-1,1,xticks=False,yticks=False)

ax=plt.subplot(423)
y_fit = (np.log10(jQ_fit)- ymin[2])/ymax[2]
y_truth = (np.log10(jQ)- ymin[2])/ymax[2]
plot_scatter(ax,y_fit,y_truth,"Fit", "Truth",r"$j_{\rm Q}$",-1,1,xticks=False)

ax=plt.subplot(424)
y_fit = (np.log10(aQ_fit)- ymin[3])/ymax[3]
y_truth = (np.log10(aQ)- ymin[3])/ymax[3]
plot_scatter(ax,y_fit,y_truth,"Fit", "Truth",r"$\alpha_{\rm Q}$",-1,1,xticks=False,yticks=False)

ax=plt.subplot(425)
plot_scatter(ax,-rQ_fit/6e-13,rQ/6e-13,"Fit", "Truth",r"$\rho_{\rm Q}/6\times10^{-13}$",-1,0.1,xticks=False)

ax=plt.subplot(426)
y_fit = (np.log10(jV_fit)- ymin[5])/ymax[5]
y_truth = (np.log10(jV)- ymin[5])/ymax[5]
plot_scatter(ax,y_fit,y_truth ,"Fit", "Truth",r"$j_{\rm V}$",-1,1,xticks=False,yticks=False)

ax=plt.subplot(427)
y_fit = (np.log10(aV_fit)- ymin[6])/ymax[6]
y_truth = (np.log10(aV)- ymin[6])/ymax[6]
plot_scatter(ax,y_fit,y_truth,"Fit", "Truth",r"$\alpha_{\rm V}$",-1,1,xticks=True)

ax=plt.subplot(428)
plot_scatter(ax,rV_fit/1e-12,rV/1e-12,"Fit", "Truth",r"$\rho_{\rm V}/10^{-12}$",-1,1,xticks=True,yticks=False)

plt.tight_layout()

plt.savefig("fit.png",dpi=100)